In [22]:
import json
import pdb
from collections import OrderedDict
import random
import pdb

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from scipy.sparse import vstack

from brick_parser import tagsetList as tagset_list, \
                         tagList as tag_list,\
                         pointTagsetList as point_tagset_list
#from imp import reload
#reload(building_tokenizer)
from building_tokenizer import get_unit_dict, get_bacnettype_dict
tagset_list = list(set(tagset_list))

In [23]:
# Note:
# May need to remove "unknown" data points from learning samples?

In [24]:
building_name = 'ebu3b'
learning_sample_num = 50
unit_dict = get_unit_dict(building_name)
bacnettype_dict = get_bacnettype_dict(building_name)
with open('metadata/{0}_ground_truth.json'.format(building_name), 'r') as fp:
    truth_dict = json.load(fp)

In [25]:
def leave_one_word(s, w):
    if w in s:
        s = s.replace(w, '')
        s = w + '-' + s
    return s
def _bilou_phraser(sentence, token_labels):                                     
    phrase_labels = list()                                                  
    curr_phrase = ''                                                        
    for i, (c, label) in enumerate(zip(sentence, token_labels)):
        """
        if 'right_identifier' in label:
            if 'right_identifier' not in curr_phrase and curr_phrase!='':
                phrase_labels.append(curr_phrase)                           
                curr_phrase = ''
            curr_phrase += 'right_identifier'+c
            continue
        elif 'left_identifier' in label:
            #pdb.set_trace()
            if 'left_identifier' not in curr_phrase and curr_phrase!='':
                phrase_labels.append(curr_phrase)
                curr_phrase = ''
            curr_phrase += 'left_identifier'+c
            continue
        """
        if label[2:] in ['right_identifier', 'left_identifier']:
            continue
        tag = label[0]                                                      
        if tag=='B':                                                        
            if curr_phrase:                                                 
            # Below is redundant if the other tags handles correctly.       
                phrase_labels.append(curr_phrase)                           
            curr_phrase = label[2:]                                         
        elif tag=='I':                                                      
            if curr_phrase != label[2:]:                                    
                phrase_labels.append(curr_phrase)                           
                curr_phrase = label[2:]                                     
        elif tag=='L':                                                      
            if curr_phrase != label[2:]:                                    
                # Add if the previous label is different                    
                phrase_labels.append(curr_phrase)                           
            # Add current label                                             
            phrase_labels.append(label[2:])                                 
            curr_phrase = ''                                                
        elif tag=='O':                                                      
            # Do nothing other than pushing the previous label              
            if curr_phrase:                                                 
                phrase_labels.append(curr_phrase)                           
            curr_phrase = ''                                                
        elif tag=='U':                                                      
            if curr_phrase:                                                 
                phrase_labels.append(curr_phrase)                           
            phrase_labels.append(label[2:])                                 
        else:                                                               
            print('Tag is incorrect in: {0}.'.format(label))                
            try:                                                            
                assert(False)                                               
            except:                                                         
                pdb.set_trace()
    if curr_phrase!='':
        phrase_labels.append(curr_phrase)
    phrase_labels = [leave_one_word(leave_one_word(phrase_label, 'left_identifier'), 'right_identifier')\
                     for phrase_label in phrase_labels]
    return phrase_labels

In [26]:
source_building = 'ebu3b'
target_building = 'ebu3b'

N = 100
token_type = 'justseparate'
label_type = 'label'
use_cluster_flag = True

# This part should be changed to use MongoDB.
label_filename = 'result/test_result_{0}_{1}_{2}_{3}_{4}_{5}.json'\
            .format(source_building, target_building, token_type, label_type, N,\
                    'clustered' if use_cluster_flag else 'unclustered')
with open(label_filename, 'r') as fp:
    word_label_dict = json.load(fp)

In [27]:
orig_phrase_dict = OrderedDict()
pred_phrase_dict = OrderedDict()
for srcid, labels in word_label_dict.items():
    orig_phrase_dict[srcid] = _bilou_phraser(labels['sentence'], labels['orig_token_labels'])
    pred_phrase_dict[srcid] = _bilou_phraser(labels['sentence'], labels['pred_token_labels'])

In [28]:
def splitter(s):
    return s.split('_')
point_tagset_list = list(map(splitter, point_tagset_list))

In [29]:
vectorizer = TfidfVectorizer(ngram_range=(1,2))
doc = [' '.join(phrases) for phrases in orig_phrase_dict.values()]
vect_doc = vectorizer.fit_transform(doc)
srcid_list = list(orig_phrase_dict.keys())

In [30]:
len(srcid_list)==len(vect_doc.toarray())

True

In [31]:
sample_srcids = random.sample(srcid_list, learning_sample_num)

In [32]:
# Find undefined tagsets (names)
undefined_tagsets = set()
for srcid in sample_srcids:
    truths = truth_dict[srcid]
    for truth in truths:
        if truth not in tagset_list:
            undefined_tagsets.add(truth)
tagset_list.extend(list(undefined_tagsets))

In [33]:
truth_mat = list()
feature_mat = list()
for srcid in sample_srcids:
    truths = truth_dict[srcid]
    truth_vector = [1 if tagset in truths else 0 for tagset in tagset_list]
    truth_mat.append(truth_vector)
    feature_mat.append(vect_doc[srcid_list.index(srcid)])
feature_mat = vstack(feature_mat)

In [34]:
learner = RandomForestClassifier()
learner.fit(feature_mat, truth_mat)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [35]:
srcid = random.sample(srcid_list, 1)[0]
assert(srcid not in sample_srcids)

In [36]:
sample_features = vect_doc[srcid_list.index(srcid)]
pred = learner.predict(sample_features)

In [37]:
indices = [i for i, v in enumerate(pred[0]) if v==1]
[tagset_list[idx] for idx in indices]

['vav',
 'effective_heating_temperature_setpoint',
 'room',
 'network_adapter-nae',
 'building-ebu3b',
 'network_adapter-n']

In [39]:
truth_dict[srcid]

['effective_heating_temperature_setpoint',
 'room',
 'building-ebu3b',
 'network_adapter-nae',
 'network_adapter-n',
 'vav']